In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import random
import os
from fake_useragent import UserAgent
import undetected_chromedriver as uc
import pickle

In [6]:

def save_cookies(driver, path):
    """Save cookies to file to avoid login next time"""
    with open(path, 'wb') as filehandler:
        pickle.dump(driver.get_cookies(), filehandler)

def load_cookies(driver, path):
    """Load cookies from file"""
    if os.path.exists(path):
        with open(path, 'rb') as cookiesfile:
            cookies = pickle.load(cookiesfile)
            for cookie in cookies:
                if 'expiry' in cookie:
                    del cookie['expiry']
                driver.add_cookie(cookie)
        return True
    return False

def random_sleep(min_seconds=2, max_seconds=5):
    """Sleep for a random time between min and max seconds"""
    time.sleep(random.uniform(min_seconds, max_seconds))

def youtube_studio_scraper(email, password, video_id):
    """
    Scrape YouTube Studio page while avoiding detection
    
    Args:
        email: YouTube account email
        password: YouTube account password
        video_id: ID of the video to access copyright info
    """
    # Create a UserAgent object for random realistic user agents
    ua = UserAgent()
    user_agent = ua.random
    
    # Set up undetected-chromedriver (better anti-detection)
    options = uc.ChromeOptions()
    options.add_argument(f'--user-agent={user_agent}')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-extensions')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-infobars')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-browser-side-navigation')
    options.add_argument('--disable-gpu')
    
    # Add proxy if needed (uncomment and configure if you want to use a proxy)
    # options.add_argument('--proxy-server=ip:port')
    
    # Create a driver with undetected-chromedriver
    driver = uc.Chrome(options=options)
    
    try:
        # Set window size to a common resolution
        driver.set_window_size(1920, 1080)
        
        # Add some entropy to mouse movements and execution
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
        # First navigate to YouTube to set cookies domain
        driver.get("https://www.youtube.com")
        random_sleep(3, 6)
        
        # Try to load cookies if available
        cookies_loaded = load_cookies(driver, f"{email}_cookies.pkl")
        
        if not cookies_loaded:
            # Login process
            driver.get("https://accounts.google.com/signin")
            random_sleep()
            
            # Enter email
            email_field = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.ID, "identifierId"))
            )
            
            # Type slowly like a human
            for char in email:
                email_field.send_keys(char)
                time.sleep(random.uniform(0.05, 0.2))
            
            random_sleep(1, 3)
            
            # Click next
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "#identifierNext button"))
            ).click()
            
            random_sleep(3, 5)
            
            # Enter password
            password_field = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='password']"))
            )
            
            # Type slowly like a human
            for char in password:
                password_field.send_keys(char)
                time.sleep(random.uniform(0.05, 0.2))
                
            random_sleep(1, 3)
            
            # Click next
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "#passwordNext button"))
            ).click()
            
            # Wait for login to complete
            random_sleep(5, 8)
            
            # Save cookies for future use
            save_cookies(driver, f"{email}_cookies.pkl")
        
        # Navigate to the target page
        target_url = f"https://studio.youtube.com/video/{video_id}/copyright"
        driver.get(target_url)
        
        # Wait for page to load completely
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        
        # Execute some random scrolls to mimic human behavior
        for _ in range(random.randint(1, 3)):
            driver.execute_script(f"window.scrollTo(0, {random.randint(100, 300)});")
            random_sleep(0.5, 2)
        
        # Additional wait to ensure the page is fully loaded
        random_sleep(5, 8)
        
        # Get the page source
        html_content = driver.page_source
        
        # Save to file
        with open(f"youtube_studio_{video_id}_copyright.html", "w", encoding="utf-8") as f:
            f.write(html_content)
        
        print(f"Successfully saved copyright info for video {video_id}")
        return html_content
        
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
        
    finally:
        # Clean exit
        random_sleep(2, 4)
        driver.quit()

if __name__ == "__main__":
    # Replace with your actual credentials and video ID
    email = "ruslan0gumerov@gmail.com"
    password = "NeBuhayTreniruysa0@"
    video_id = "yCQ3iQ8FxMw"  # The video ID from your URL
    
    youtube_studio_scraper(email, password, video_id)

An error occurred: 'NoneType' object has no attribute 'is_displayed'
